In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from os import listdir
from os import listdir, mkdir
from sklearn.decomposition import PCA
from numpy import asarray
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

# Generating User Data for Registeration

In [ ]:
facecascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [ ]:
# Function that creates a directory
def create_directory(directory_name, path_to_directory):
    # Path 
    path = os.path.join(path_to_directory, directory_name) 
    
    mkdir(path) 
    return(print('Done!'))

In [ ]:
# Directory Name 
directory = "imagebucket"

# Parent Directory path 
parent_dir = "/Users/Kenechi/Artifical Intelligence/Project Vindler/FaceRecognition-Vindler/"

# Create the directory
create_directory(directory , parent_dir)

In [ ]:
# Function to create user faceimage data
def generate_dataset(image, p_id, image_id):
    cv2.imwrite(f'{directory}/user.'+str(p_id)+'.'+str(image_id)+'.jpg',image)

In [ ]:
# function to draw boundry around face in image
def draw_boundary(image, classifier, scalefactor, min_neighbors, color, text):
    gray_scale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_scale, scalefactor, min_neighbors)
    coordinates = []
    
    for (x, y, w, h) in features:
        cv2.rectangle(image, (x,y),(x+w, y+h), color, 1)        
        cv2.putText(image, text, (x, y-4), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2, cv2.LINE_AA)
        coordinates = [x, y, w, h]
        
    return coordinates

In [ ]:
# function to obtain face boundry in image

def detect(image, facecascade, image_id, p_id ):
    color = {'blue':(0,0,255), 'red':(255,0,0), 'green':(0,255,0), 'white':(255,255,255)}
    coordinate = draw_boundary(image, facecascade, 1.1, 10, color['white'], 'Detecting Face')
    
    if len(coordinate)== 4:
        area_of_focus = image[coordinate[1]:coordinate[1]+coordinate[3], coordinate[0]:coordinate[0]+coordinate[2]]
               
        #p_id = 1
        generate_dataset(area_of_focus, p_id, image_id)
    return image

In [ ]:
def collect_data(p_id):
    
    print('PLEASE LOOK INTO THE SCREEN TILL IT DISAPPEARS.')
   
    video_capture  = cv2.VideoCapture(0) #if using an external webcam type in -1 inplace of 0
    image_id = 0

    while True:
        _ ,image = video_capture.read()
        image = detect(image, facecascade, image_id, p_id)
        cv2.imshow('Collecting Face Images for Registration', image)
        image_id += 1
        if image_id == 300:#cv2.waitKey(1) & 0xFF == ord('e'): # to break loop press e
            break

    video_capture.release()
    cv2.destroyAllWindows()
    return(print('Thank You!'))

In [ ]:
p_id=int(input('Type in your ID'))

In [ ]:
collect_data(p_id)

### TRAINING THE MODEL

In [ ]:
# function that creates features and targets
def load_train_images(directory):
    train_pictures = list()
    targets = list()
    
    print('Converting to numpy array ...')
    for picture in listdir(directory) :
         if picture != 'Thumbs.db':
                 # load the picture from directory
                photo = load_img(directory + picture, target_size = (224,224))
                
                #convert image to numpy array
                photo = img_to_array(photo, dtype='uint8')  
                
                # append to list
                train_pictures.append(photo)
                
                
                # label targets
                targets.append(picture.split('.')[1])
                
                
    print(' ')
    
    X = asarray(train_pictures, dtype = 'uint8')
    Y = asarray(targets, dtype = 'uint8')
    
    X_r = np.array(X).reshape((300, 224*224))
    
    pca_t = PCA()
    pca_t.fit(X_r)
    
    explained_variance = pca_t.explained_variance_ratio
    cumsum = np.cumsum(explained_variance)
    n_comp = np.argmax(cumsum1 >= 0.99) + 1 
    
    pca_m= PCA(n_components = n_comp)
    x = pca_m.fit(X_r)
    
    Fclassifier = SVC(class_weight='balanced',C= 1, kernel= 'linear')
    
    Fclassifier.partial_fit(x_train, y_train) # HERE IS THE PARTIAL FIT SHENANIGAN!
    

    
    print(' ')
    print('Done!')
    return Fclassifier

In [ ]:
Fclassifier = load_train_images(directory)

### PREDICTIONS

In [ ]:
def draw_boundary(image, classifier, scalefactor, min_neighbors, color, text, Fclassifier):
    gray_scale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_scale, scalefactor, min_neighbors)
    coordinates = []
    
    for (x, y, w, h) in features:
        cv2.rectangle(image, (x,y),(x+w, y+h), color, 1)
        id_, _= Fclassifier.predict(gray_scale[y:y+h, x:x+w])
        
        if id_ == p_id:
            cv2.putText(image, 'Hello User', (x, y-4), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2, cv2.LINE_AA)
            coordinates = [x, y, w, h]
        
    return coordinates

In [ ]:
def recognize(image, Fclassifier,facecascade ):
    color = {'blue':(0,0,255), 'red':(255,0,0), 'green':(0,255,0), 'white':(255,255,255)}
    coordinate = draw_boundary(image,facecascade,1.1, 10, color['white'], 'FACE', Fclassifier)
    return image

In [ ]:
def resize(image):
    height = 224
    width = 224
    dim = (width, height)
    image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    return image
  

In [ ]:
def grant_access():
    auth_count = 0

    while True:
        _ ,image = video_capture.read()

        image = resize(image)

        #image = detect(image, facecascade, image_id)
        image = recognize(image, Fclassifier, facecascade)
        cv2.imshow('Approved', image)
        auth_count += 1
        if auth_count == 6:#cv2.waitKey(1) & 0xFF == ord('e'): # to break loop press e
            break

    video_capture.release()
    cv2.destroyAllWindows()
    return('Done!')

In [ ]:
grant_access()